In [44]:
import pandas as pd
import numpy as np
import csv
import math
from bs4 import BeautifulSoup
from gensim.summarization.bm25 import get_bm25_weights
from rank_bm25 import BM25Okapi
import json

# Read the json file from kevin

In [ ]:
data_df_layer_1 = pd.read_json('../output_data/layer_1_output_10.json')

# Find the pair of unique phrase in layer1

In [ ]:
unique_phrase_list = np.unique(data_df_layer_1['phrase'].values)

In [ ]:
unique_phrase_list

# Find the unique phrase in layer 1

In [ ]:
p_list = list(pd.read_json('./top_k_output/layer_1.json',typ='series').keys())

In [ ]:
len(p_list)

# Create list of list(unique phrase) for current pair

In [ ]:
#loop throught every pair of phrase
list_of_phrase_list = []
for i in range(len(unique_phrase_list)):
    tmplist =[]
    for j in range(len(p_list)):
        if p_list[j] in unique_phrase_list[i]:
            tmplist.append(p_list[j])
    list_of_phrase_list.append(tmplist)

# attention: Every sentence in the articles from the pool is a candidate sentence for the final summary.
# step: compute a BM25 score for every candidate sentence

# to compute BM25
# 1. find all sentence in current layer(done)
# 2. index all sentence(done)
# 3. compute score for all current sentence
# 4. question here-> should we use the sentence with highest score 
# to cover the phrase? or to cover pair of phrase?
# If so, after phrase been recoverd, tag the sentence been used,
# recompute the BM25 in unused sentence pool
# and iterate all sentece until all phrase been coverd

In [ ]:
#create the article pool now
article_list = data_df_layer_1['article'].values
#for each article, find all sentence
article_list[0]
sentence_dic = {}
list_sentence = []
s_count = 0 #sentence index
for i in range(len(article_list)):
    #for every sentence, if not in sentence_list, push sentence in list
    tmp_sentence_list = article_list[i].split(".")
    for j in range(len(tmp_sentence_list)):
        if tmp_sentence_list[j] not in article_list:
            sentence_dic[s_count] = tmp_sentence_list[j]
            list_sentence.append(tmp_sentence_list[j])
            s_count +=1

In [ ]:
list_sentence = np.unique(list_sentence)

In [ ]:
len(list_sentence)

In [40]:
def BM25_score(sentence_list,unique_phrase_list,p_list):
    #at each iteration
    #find the sentence that could lead to the highest bm25
        #1.for every pair of phrase, find the max score of that sentence, save that score and the respective sentence,
        #2.pick the highest score, also recover that sentence
        #3.find how many phrase this sentece been touched, tag those phrase, pop those phrase out of phrase list
        #4.if pair of phrase that lead to highest score contains all some at least two of phrase that been poped out,
        #5.pop out that phrase pair in pair list
        #6.finsihed current iteration
        #7.check whether the lenght of phrass list becomes 0 or num of iteration hit max
        #8.if either happens, exit the problem, return the sentence list, and the touched phrase list
        
    
    answer_sentence_list = []
    touched_phrase_list = []
    count = 0
    while len(p_list) > 0 and len(touched_phrase_list) < len(p_list):
        #step1
        #create a data structure to save score for current query(as pair of phrase)
        #in the future, implement the max-heap ds here O(nlogn) push, O(1) peek
        score_list_current_iter = []
        sentence_idx_list = []
        cur_pair_phrase_dic = {}
        for i in range(len(unique_phrase_list)):
            #compute bm25 score use current phrase pair as query
            bm25 = BM25Okapi(sentence_list) #create class of bm25
            doc_scores = bm25.get_scores(unique_phrase_list[i])
            sentence_loc = np.argmax(doc_scores)  #the index num of max-score sentence in the sen_list
            sentence_score = np.max(doc_scores)   #the max score over all score of sentence for current query
            score_list_current_iter.append(sentence_score)
            sentence_idx_list.append(sentence_loc)
            if sentence_score not in cur_pair_phrase_dic:
                cur_pair_phrase_dic[sentence_score] = i
        #step2
        highest_score_index = np.argmax(score_list_current_iter)
        high_sen = sentence_list[sentence_idx_list[highest_score_index]] 
        answer_sentence_list.append(high_sen)
        #step three
        cur_touched_phrase = []  #record how many phrase been touched by this sentence
#         print('current lengh of p_list is', len(p_list))
        list_deleted = []
        for pos in range(len(p_list)):
#             print('current pos is', pos)
#             print('what is that ', p_list[pos])
            if p_list[pos] in high_sen:
                if p_list[pos] not in cur_touched_phrase:
                    cur_touched_phrase.append(p_list[pos])
#                 p_list.remove(p_list[pos])
                #remove phrase list
        #<wait>
#         for i in range(len(cur_touched_phrase)):
#             if cur_touched_phrase[i] in p_list:
#                 p_list.remove(cur_touched_phrase[i])
        #<wait>
#         print('current sentence is', sentence_list[highest_score_index])
#         print('len of unique pair  list', len(unique_phrase_list))
        print('lengh of touched phrase', len(cur_touched_phrase))
#         print('current count is', count)
#         print('lengh of sentence list', len(sentence_list))

#         #if the current pair of phrase that lead to this sentence which has max score contains two phrase in cur_touched_phrase
#         #pop this pair of phrase out of pair of phrase list
#         #return the pair of phrase by useing dic
        curpair = unique_phrase_list[cur_pair_phrase_dic[np.max(score_list_current_iter)]] 
#         #count how many phrase in cur_touched_phrase 
        count_now = 0
        for loc in range(len(cur_touched_phrase)):
            if cur_touched_phrase[loc] in curpair:
                count_now+=1
        if count_now>=2 and curpair in unique_phrase_list:
#             print('hahahahahahhaa six six six')
            unique_phrase_list.remove(curpair)
        count +=1
        #remove phrase list
        #unique_phrase_list.remove(curpair)
        #remove sentence
        sentence_list.remove(high_sen)
        
        #add cur touched list to total touched list
        for inx in range(len(cur_touched_phrase)):
            if cur_touched_phrase[inx] not in touched_phrase_list:
                touched_phrase_list.append(cur_touched_phrase[inx])
        print('now total touched length', len(touched_phrase_list))
        #create a pair phrase function based on the cur_touched_phrase_process
        tmp_phrase_pair = []
        for p in range(len(cur_touched_phrase)):
            for q in range(p,len(cur_touched_phrase)):
                if cur_touched_phrase[p] != cur_touched_phrase[q]:
                    tmp_phrase_pair.append([cur_touched_phrase[p],cur_touched_phrase[q]])
        #iter thorught unique_phrase_list if pair is exist in touched list, dequeue them
        
        deletelist = []
#         print('tmo coutched list is', len(tmp_phrase_pair))
#         print('tmp phrase pari look like ', tmp_phrase_pair)
        for p2 in range(len(tmp_phrase_pair)):
            for q2 in range(len(unique_phrase_list)):
                if tmp_phrase_pair[p2][0] in unique_phrase_list[q2] and tmp_phrase_pair[p2][1] in unique_phrase_list[q2]:
                    if unique_phrase_list[q2] not in deletelist:
                        deletelist.append(unique_phrase_list[q2])
        
#         print('look of deletelist, ', deletelist)
#         print('len of delete list', len(deletelist))
        for p3 in range(len(deletelist)):
            if deletelist[p3] in unique_phrase_list:
                unique_phrase_list.remove(deletelist[p3])
#         print('len of unique list is', len(unique_phrase_list))
        print('num of iteration now is', count)

    return answer_sentence_list

In [41]:
answer = BM25_score(list(list_sentence),list(unique_phrase_list),p_list.copy())

lengh of touched phrase 2
now total touched length 2
num of iteration now is 1
lengh of touched phrase 1
now total touched length 3
num of iteration now is 2
lengh of touched phrase 0
now total touched length 3
num of iteration now is 3
lengh of touched phrase 0
now total touched length 3
num of iteration now is 4
lengh of touched phrase 0
now total touched length 3
num of iteration now is 5
lengh of touched phrase 0
now total touched length 3
num of iteration now is 6
lengh of touched phrase 0
now total touched length 3
num of iteration now is 7
lengh of touched phrase 0
now total touched length 3
num of iteration now is 8
lengh of touched phrase 1
now total touched length 4
num of iteration now is 9
lengh of touched phrase 0
now total touched length 4
num of iteration now is 10
lengh of touched phrase 0
now total touched length 4
num of iteration now is 11
lengh of touched phrase 1
now total touched length 4
num of iteration now is 12
lengh of touched phrase 0
now total touched lengt

lengh of touched phrase 0
now total touched length 9
num of iteration now is 106
lengh of touched phrase 0
now total touched length 9
num of iteration now is 107
lengh of touched phrase 1
now total touched length 9
num of iteration now is 108
lengh of touched phrase 1
now total touched length 9
num of iteration now is 109
lengh of touched phrase 0
now total touched length 9
num of iteration now is 110
lengh of touched phrase 1
now total touched length 9
num of iteration now is 111
lengh of touched phrase 1
now total touched length 9
num of iteration now is 112
lengh of touched phrase 0
now total touched length 9
num of iteration now is 113
lengh of touched phrase 0
now total touched length 9
num of iteration now is 114
lengh of touched phrase 2
now total touched length 9
num of iteration now is 115
lengh of touched phrase 2
now total touched length 9
num of iteration now is 116
lengh of touched phrase 1
now total touched length 9
num of iteration now is 117
lengh of touched phrase 0
no

In [42]:
answer

['1 A resolution and its implications in the mechanism of water oxidatio Oxygen-evolving complex of photosystem II (PSII) is a tetra-manganese calcium penta-oxygenic cluster (Mn4CaO5) catalyzing light-induced water oxidation through several intermediate states (S-states) by a mechanism that is not fully understood',
 'The dynamic Intensity-Weighted Region of Interest (dIWROI) technique spatially varies beam intensity during acquisition with copper filter collimation',
 ' In comparison with Gr II, the healthy children showed relatively homogenous TEG values with a tendency to hypercoagulability; the maximum was found in age group 1-3 months, decreasing towards adult values in the course of the first year of life',
 ' To improve calibration of the prognostic models, a first-level customization was performed, using logistic regression on the original scores, and the corresponding probability of ICU death was calculated for the customized scores (C-SAPS II, C-SAPS 3, and C-APACHE II)',
 ' 

In [45]:
with open('./2d_output/layer_1_output.json', 'w') as outfile:
    json.dump(answer, outfile)